In [1]:
import string

def process_file(filename, skip_header):
    """Makes a histogram that contains the words from a file.

    filename: string
    skip_header: boolean, whether to skip the Gutenberg header
   
    Returns: map from each word to the number of times it appears.
    """
    hist = {}
    fp = file(filename)

    if skip_header:
        skip_gutenberg_header(fp)

    for line in fp:
        #print line
        process_line(line, hist)
    return hist


def skip_gutenberg_header(fp):
    """Reads from fp until it finds the line that ends the header.

    fp: open file object
    """
    for line in fp:
        if line.startswith('*END*THE SMALL PRINT!'):
            break


def process_line(line, hist):
    """Adds the words in the line to the histogram.

    Modifies hist.

    line: string
    hist: histogram (map from word to frequency)
    """
    # replace hyphens with spaces before splitting
    line = line.replace('-', ' ')
    
    for word in line.split():
        # remove punctuation and convert to lowercase
        word = word.strip(string.punctuation + string.whitespace)
        word = word.lower()

        # update the histogram
        hist[word] = hist.get(word, 0) + 1

In [2]:
hist = process_file('jane_austen_pride_and_prejudice.txt', False)

In [3]:
def word_frequency(hist):
    #count number of words and the frequency for each word
    #the hist keys are the unique words, and is values are the number of times it is used
    numberofwords, uniquewords = 0, 0
    for word, val in hist.iteritems():
        uniquewords += 1
        numberofwords += val
    return (uniquewords, numberofwords)

In [4]:
uniquewords, numberofwords = word_frequency(hist)
print uniquewords, numberofwords

6729 125324


In [5]:
def top_20_words(hist):
    #prints the 20 most used words
    #maps from hist (key=word,value=freq) to list [((freq,word))]
    word_list = []
    for word, count in hist.iteritems():
        word_list.append((count,word))
    word_list.sort(reverse=True)
    for count, word in word_list[:20]:
        print word, count

In [6]:
top_20_words(hist)

the 4506
to 4243
of 3730
and 3658
her 2225
i 2069
a 2011
in 1937
was 1847
she 1710
that 1594
it 1550
not 1450
you 1427
he 1338
his 1271
be 1260
as 1192
had 1177
with 1100


In [7]:
def read_words(f):
    word_dict = {}
    fin = open(f)
    for line in fin:
        word_dict[line.strip()]=1
    return word_dict

In [8]:
known_words = read_words('words.txt')

In [9]:
def unknown_words(book, known_words):
    #compares book and known_words and assembles dictionary of words that are not in known_words
    unknown = {}
    for word in book:
        if not word in known_words:
            unknown[word] = None
    return unknown

In [10]:
unknowns = unknown_words(hist, known_words)

In [11]:
print unknowns

{'': None, 'rencontre': None, 'expostulation': None, 'expence': None, "jane's": None, 'synonymously': None, "daughter's": None, 'bakewell': None, '99712': None, "friend's": None, 'prognostics': None, 'superciliousness': None, 'apartment': None, 'essentials': None, 'unenforceability': None, 'fitzwilliam': None, 'thursday': None, 'neighbours': None, 'tm': None, 'dissatisfied': None, 'gratulation': None, 'repinings': None, 'downloading': None, 'unabashed': None, 'covies': None, "women's": None, 'chamberlayne': None, 'recollected': None, "anne's": None, 'cheapside': None, 'ceaseless': None, 'michael': None, 'p': None, 'desertion': None, 'delightfully': None, 'harriet': None, 's/he': None, 'edw': None, "kitty's": None, 'designedly': None, 'abominably': None, '596': None, 'diffuseness': None, 'honourable': None, '4557': None, "gardiner's": None, 'pemberley': None, 'richard': None, "other's": None, 'irrevocably': None, 'eliza': None, 'hatefully': None, "man's": None, "housekeeper's": None, 'c

In [12]:
import random

def make_cdf(hist):
    cdf = []
    for word, count in hist.iteritems():
#        print word
        for i in range(0,count):         
            cdf.append(word)
    return cdf

def choose_from_hist(hist):
    #picks a sample from history, with chance proportional to sample size in histogram
    #e.g. book dictionary, we know number of unique words and the frequency, i.e. total number of words
    #need a 'step'function so we can choose a random int value that alignes with the choice
    #analogy: hist is the pdf, we need a cdf!
    #map pdf to cdf
    #idea: make a list with total words entries, and accumulate the pdf in the cdf list with word x occurrence
    cdf = make_cdf(hist)
    print len(cdf)
#    print cdf[100:110]
    word = random.choice(cdf)
    return word

In [13]:
choose_from_hist(hist)

125324


'come'

In [14]:
#More elegant than make_cdf
def random_word(hist):
    """Chooses a random word from a histogram.

    The probability of each word is proportional to its frequency.
    """
    t = []
    for word, freq in hist.items():
        t.extend([word] * freq)
    print len(t)

    return random.choice(t)

In [15]:
random_word(hist)

125324


'room'

## 13.8 Markov Analysis (below wrong, is based on word length, not # of words

In [17]:
def wordfilter(filename, wordlength):
    #opens the book, start reading lines, keep memory of previous line
    #for each word, check if wordlength = filter
    fp = file(filename)
#    if skip_header:
#        skip_gutenberg_header(fp)
    prev_word = ''
    get_suffix = False
    prefix = {}
    for this_line in fp:    
        this_line = this_line.replace('-', ' ')
        for this_word in this_line.split():
            # remove punctuation and convert to lowercase
            this_word = this_word.strip(string.punctuation + string.whitespace)
            this_word = this_word.lower()
            #print this_word
            if get_suffix:
                if prev_word not in prefix:
                    prefix[prev_word] = [this_word]
                else:
                    prefix[prev_word].append(this_word)
            get_suffix = len(this_word) == wordlength
            if get_suffix:
                prev_word = this_word        
    return prefix

In [18]:
prefix = wordfilter('jane_austen_pride_and_prejudice.txt',3)

In [19]:
prefix

{'1.a': ['by'],
 '1.b': ['project'],
 '1.c': ['below', 'the'],
 '1.d': ['the'],
 '1.e': ['below', 'unless'],
 '1.f': ['1.f.1'],
 '596': ['1887'],
 '801': ['596'],
 '809': ['north'],
 'act': ['of', 'in', 'of', 'in', 'as', 'as', 'against', 'in', 'do'],
 'add': ['to',
  'something',
  'very',
  'considerably',
  'that',
  'god',
  'that',
  'aught',
  'something',
  'i',
  'a',
  'force'],
 'age': ['two',
  'i',
  'her',
  'she',
  'to',
  'since',
  'of',
  'with',
  'i',
  'are',
  'with',
  'and',
  'howsoever',
  'and',
  'did'],
 'ago': ['i',
  'i',
  'to',
  'exactly',
  'before',
  'he',
  'from',
  'and',
  'she',
  'asserted',
  'i',
  'i',
  'would',
  'and',
  'attributed',
  'the',
  'she',
  'would',
  'governess',
  'never',
  'was',
  'every',
  'oh',
  'i',
  'i',
  'i'],
 'aid': ['he'],
 'air': ['of',
  'and',
  'of',
  'and',
  'and',
  'was',
  'as',
  'all',
  'and',
  'of',
  'of',
  'for',
  'of',
  'more',
  'with',
  'of',
  'the',
  'was',
  'which',
  'and',
  'w

In [1]:
"""This module contains code from
Think Python by Allen B. Downey
http://thinkpython.com

Copyright 2012 Allen B. Downey
License: GNU GPLv3 http://www.gnu.org/licenses/gpl.html

"""

import sys
import string
import random

# global variables
suffix_map = {}        # map from prefixes to a list of suffixes
prefix = ()            # current tuple of words


def process_file(filename, order=2):
    """Reads a file and performs Markov analysis.

    filename: string
    order: integer number of words in the prefix

    Returns: map from prefix to list of possible suffixes.
    """
    fp = open(filename)
    skip_gutenberg_header(fp)

    for line in fp:
        for word in line.rstrip().split():
            process_word(word, order)


def skip_gutenberg_header(fp):
    """Reads from fp until it finds the line that ends the header.

    fp: open file object
    """
    for line in fp:
        if line.startswith('*END*THE SMALL PRINT!'):
            break


def process_word(word, order=2):
    """Processes each word.

    word: string
    order: integer

    During the first few iterations, all we do is store up the words; 
    after that we start adding entries to the dictionary.
    """
    global prefix
    if len(prefix) < order:
        prefix += (word,)
        return

    try:
        suffix_map[prefix].append(word)
    except KeyError:
        # if there is no entry for this prefix, make one
        suffix_map[prefix] = [word]


    prefix = shift(prefix, word)


def random_text(n=100):
    """Generates random wordsfrom the analyzed text.

    Starts with a random prefix from the dictionary.

    n: number of words to generate
    """
    # choose a random prefix (not weighted by frequency)
    start = random.choice(suffix_map.keys())
    
    for i in range(n):
        suffixes = suffix_map.get(start, None)
        if suffixes == None:
            # if the start isn't in map, we got to the end of the
            # original text, so we have to start again.
            random_text(n-i)
            return
        #print suffixes
        # choose a random suffix
        word = random.choice(suffixes)
        print word,
        start = shift(start, word)
        #print start


def shift(t, word):
    """Forms a new tuple by removing the head and adding word to the tail.

    t: tuple of strings
    word: string

    Returns: tuple of strings
    """
    return t[1:] + (word,)


def main(name, filename='', n=100, order=2, *args):
    try:
        n = int(n)
        order = int(order)
    except:
        print 'Usage: randomtext.py filename [# of words] [prefix length]'
    else: 
        process_file(filename, order)
        random_text(n)

In [2]:
main('emma.txt','emma.txt', 100, 3)

unfavourable likeness, to every morning visitor in Brunswick Square;--and, as I said, I did then forswear ever drawing any body again. But for Harriet's sake, or rather for my own, and as there are no husbands and wives in the case _at_ _present_, I will break my resolution now." Mr. Elton seemed very properly struck and delighted by the idea, and was repeating, "No husbands and wives in the case at present indeed, as you observe. Exactly so. No husbands and wives," with so interesting a consciousness, that Emma began to feel she had been saying to Mrs. Elton, and


In [52]:
import matplotlib.pyplot as plt
import math

def read_book(book):
    hist = {}
    fp = file(book)
    for this_line in fp:    
        this_line = this_line.replace('-', ' ')
        for this_word in this_line.split():
            # remove punctuation and convert to lowercase
            this_word = this_word.strip(string.punctuation + string.whitespace)
            this_word = this_word.lower()
            hist[this_word] = hist.get(this_word, 0) + 1
    return hist

def create_rank(hist):
    wordfreq = hist.values()
    wordfreq.sort(reverse=True)
    # enumerate the ranks and frequencies 
    rf = [(r+1, f) for r, f in enumerate(wordfreq)]
    #print rf
    return rf

def print_rank(rf):
    for r, f in rf:
        #print math.log(r), math.log(f)
        print r, f
    return

def plot_rank(rf, scale='log'):
    r, f = zip(*rf)
    plt.xscale(scale)
    plt.yscale(scale)
    plt.title('Zipf plot')
    plt.xlabel('rank')
    plt.ylabel('frequency')
    plt.plot(r, f, 'ro')
    plt.show()
    

def doit():
    hist = read_book('emma.txt')
    create_rank(hist)
    rf = create_rank(hist)
    plot_rank(rf)

In [53]:
doit()